In [9]:
! git clone https://github.com/iolkhovsky/retinanet_torch
%cd retinanet_torch/
! pwd
! ls

Cloning into 'retinanet_torch'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (336/336), done.
remote: Total 362 (delta 224), reused 158 (delta 20), pack-reused 0
Receiving objects: 100% (362/362), 13.89 MiB | 36.11 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/retinanet_torch/retinanet_torch
/content/retinanet_torch/retinanet_torch
configs  docs	 requirements.txt  sandbox.ipynb  tests		      train.py
dataset  models  retinanet.ipynb   test_images	  train_lightning.py  utils


In [10]:
! git pull origin develop

From https://github.com/iolkhovsky/retinanet_torch
 * branch            develop    -> FETCH_HEAD
Already up to date.


In [11]:
%%capture
! pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

In [12]:
!pip install tensorboard --upgrade

Requirement already up-to-date: tensorboard in /usr/local/lib/python3.7/dist-packages (2.4.1)


In [13]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<IPython.core.display.Javascript object>

In [14]:
import sys
from os.path import join
from pathlib import Path

parent_dir = str(Path().parent.absolute())
sys.path.insert(0, parent_dir)
print(sys.path)

['/content/retinanet_torch/retinanet_torch', '/content/retinanet_torch', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [15]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datetime
from os.path import join, isdir
from os import makedirs

session_timestamp = str(datetime.datetime.now())
session_timestamp = session_timestamp.replace(" ", "").replace(":", "-").replace(".", "-")
colab_workspace = join("/content/drive/MyDrive", "colab_workspace")
session_storage = join(colab_workspace, session_timestamp)
makedirs(session_storage)
print(f"Session storage has bee created: {session_storage}")

checkpoints_root = join(session_storage, "checkpoints")
makedirs(checkpoints_root)
print(f"Checkpoints storage has bee created: {checkpoints_root}")
logs_root = join(session_storage, "logs")
makedirs(logs_root)
print(f"Tensorboard logs storage has bee created: {logs_root}")

dataset_storage = join(colab_workspace, "data")
if not isdir(dataset_storage):
  makedirs(dataset_storage)

Session storage has bee created: /content/drive/MyDrive/colab_workspace/2021-03-1019-50-31-661953
Checkpoints storage has bee created: /content/drive/MyDrive/colab_workspace/2021-03-1019-50-31-661953/checkpoints
Tensorboard logs storage has bee created: /content/drive/MyDrive/colab_workspace/2021-03-1019-50-31-661953/logs


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.tensorboard import SummaryWriter

from models.retinanet import RetinanetLightning

writer = SummaryWriter(log_dir=logs_root)
print(f"Writer initialized at {logs_root}")

model = RetinanetLightning(classes_cnt=21,
                           tboard_writer=writer,
                           train_batch=32,
                           val_batch=64,
                           dataset_storage=dataset_storage)
print(f"Model has been built {model}")

checkpoint_callback = ModelCheckpoint(
    join(checkpoints_root, "retinanet_voc.ckpt"),
    save_top_k=5,
    verbose=True,
    monitor='val_loss',
    mode='min'
)
print(f"Checkpoints storage: {checkpoints_root}")

trainer = pl.Trainer(max_epochs=20,
                     limit_val_batches=1,
                     #gpus=1,
                     val_check_interval=50,
                     check_val_every_n_epoch=1,
                     checkpoint_callback=checkpoint_callback)

trainer.fit(model)


Writer initialized at /content/drive/MyDrive/colab_workspace/2021-03-1019-37-36-273789/logs


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name      | Type                | Params
--------------------------------------------------
0 | model     | RetinanetMobilenet2 | 31.0 M
1 | box_coder | FasterRCNNBoxCoder  | 0     
2 | criterion | RetinaNetLoss       | 0     
--------------------------------------------------
31.0 M    Trainable params
0         Non-trainable params
31.0 M    Total params
124.076   Total estimated model params size (MB)


Model has been built Lightning_Retinanet_Mobilenetv2_6fm_21c_9a
Checkpoints storage: /content/drive/MyDrive/colab_workspace/2021-03-1019-37-36-273789/checkpoints



Extracting ../data/VOCtrainval_11-May-2012.tar to ../data
Using downloaded and verified file: ../data/VOCtrainval_11-May-2012.tar
Extracting ../data/VOCtrainval_11-May-2012.tar to ../data
